# Eat Safe, Love

## Notebook Set Up

In [76]:
# Import dependencies
import pymongo

from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [77]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [78]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [79]:
# review the collections in our database
db.list_collection_names()


['establishments']

In [80]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [81]:
# Find the establishments with a hygiene score of 20
query = {"score.Hygiene": 20}

# Find the establishments matching the query
result = establishments.find(query)

# Count the number of documents in the result
count = establishments.count_documents(query)

# Print the result
print("Number of establishments with a hygiene score of 20:", count)

Number of establishments with a hygiene score of 20: 0


In [82]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [83]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"}, "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
document = establishments.find_one(query)
pprint(document)


Number of establishments in London with RatingValue >= 4: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64b04e7e82bb94ffe142a995'),
 'geocode': {'latitude': '51.369321', 'longitude': '0.508551'},
 'links': [{'href': 'https://api.rating

In [84]:
# Convert the result to a Pandas DataFrame
result = establishments.find(query)

df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in DataFrame: 33
                        _id   FHRSID  ChangesByServerID  \
0  64b04e7e82bb94ffe142a995   621707                  0   
1  64b04e7e82bb94ffe142acc2  1130836                  0   
2  64b04e7e82bb94ffe142b7fb   293783                  0   
3  64b04e7e82bb94ffe142c612   294474                  0   
4  64b04e7e82bb94ffe142c615  1315095                  0   
5  64b04e7e82bb94ffe142cd0a   294900                  0   
6  64b04e7f82bb94ffe142ee2a   293756                  0   
7  64b04e7f82bb94ffe142f1dd   878523                  0   
8  64b04e7f82bb94ffe142f1fd   295107                  0   
9  64b04e7f82bb94ffe142f1fe   293772                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000014647              Tilbur

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [85]:
# Get the geolocation of "Penang Flavours" restaurant
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})
penang_geocode = penang_flavours["geocode"]
penang_latitude = float(penang_geocode["latitude"])
penang_longitude = float(penang_geocode["longitude"])

# Create the pipeline to find the top 5 establishments with a RatingValue of 5, sorted by lowest hygiene score, nearest to Penang Flavours
pipeline = [
    {"$match": {
        "RatingValue": 5,
        "scores.Hygiene": {"$ne": None},
        "geocode.latitude": {"$gte": penang_latitude - 0.01, "$lte": penang_latitude + 0.01},
        "geocode.longitude": {"$gte": penang_longitude - 0.01, "$lte": penang_longitude + 0.01}
    }},
    {"$sort": {"scores.Hygiene": 1}},
    {"$limit": 5}
]

# Execute the aggregation query
result = establishments.aggregate(pipeline)

# Count the number of documents in the result
count = establishments.count_documents({
    "RatingValue": 5,
    "scores.Hygiene": {"$ne": None},
    "geocode.latitude": {"$gte": penang_latitude - 0.01, "$lte": penang_latitude + 0.01},
    "geocode.longitude": {"$gte": penang_longitude - 0.01, "$lte": penang_longitude + 0.01}
})
print("Number of establishments with a RatingValue of 5, sorted by lowest hygiene score, nearest to Penang Flavours:", count)

# Print the first 10 results
print("First 10 results:")
for document in result:
    print(document)

Number of establishments with a RatingValue of 5, sorted by lowest hygiene score, nearest to Penang Flavours: 0
First 10 results:


In [86]:
# Convert result to Pandas DataFrame
print(f"Number of rows in DataFrame: {count}")
print(df.head(10))

Number of rows in DataFrame: 0
                        _id   FHRSID  ChangesByServerID  \
0  64b04e7e82bb94ffe142a995   621707                  0   
1  64b04e7e82bb94ffe142acc2  1130836                  0   
2  64b04e7e82bb94ffe142b7fb   293783                  0   
3  64b04e7e82bb94ffe142c612   294474                  0   
4  64b04e7e82bb94ffe142c615  1315095                  0   
5  64b04e7e82bb94ffe142cd0a   294900                  0   
6  64b04e7f82bb94ffe142ee2a   293756                  0   
7  64b04e7f82bb94ffe142f1dd   878523                  0   
8  64b04e7f82bb94ffe142f1fd   295107                  0   
9  64b04e7f82bb94ffe142f1fe   293772                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000014647              Tilbury

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [87]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
result = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
count = establishments.count_documents({"score.Hygiene": 0})
print("Number of establishments with a hygiene score of 0:", count)

# Print the first 10 results
print("First 10 results:")
for document in result[:10]:
    print(document)

Number of establishments with a hygiene score of 0: 0
First 10 results:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [88]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print("Number of rows in DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of DataFrame:")
print(df.head(10))

Number of rows in DataFrame: 55
First 10 rows of DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
